In [1]:
!pip install category-encoders

     |████████████████████████████████| 82 kB 1.0 MB/s eta 0:00:01


In [2]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

df = pd.read_csv("data.csv")

df.head()

,Unnamed: 0,date_time,zone_id,banner_id,campaign_clicks,os_id,country_id,clicks,day,hour,weekday,is_holiday,days_ago
0,0,2021-09-27 00:01:30,0,0,0,0,0,1,27,0,0,0,6
1,1,2021-09-30 00:03:59,27,0,0,0,0,1,30,0,3,0,6
2,2,2021-09-30 00:04:44,27,0,0,0,0,1,30,0,3,0,6
3,3,2021-09-27 00:09:07,1,0,0,4,0,1,27,0,0,0,6
4,4,2021-09-30 00:05:17,27,0,0,0,0,1,30,0,3,0,6


Дропаем колонки-о-которых-нельзя-говорить

In [2]:
df = df.drop(['oaid_hash', 'banner_id0', 'banner_id1', 'rate0', 'rate1', 'g0', 'g1', 'coeff_sum0', 'coeff_sum1'], axis=1)

Добавим немного банальных,  но важных фичей о датах:

In [3]:
df['date_time'] = df['date_time'].progress_apply(pd.to_datetime)

100%|██████████| 15821472/15821472 [17:33<00:00, 15024.34it/s]


In [ ]:
df['day'] = df['date_time'].progress_apply(lambda x: x.day)
df['hour'] = df['date_time'].progress_apply(lambda x: x.hour)

In [37]:
df['weekday'] = df['date_time'].progress_apply(lambda x: x.weekday())
df['is_holiday'] = df['weekday'].progress_apply(lambda x: 1 if x in [5, 6] else 0)


100%|██████████| 15821472/15821472 [00:21<00:00, 736023.82it/s]

Важная фича - добавим возраст баннера, чтобы модель могла отличать его от новых баннеров

In [5]:
first_banner_appearence = df.sort_values(by='date_time').groupby('banner_id').first()['day'].reset_index()
first_banner_appearence['days_ago'] = (first_banner_appearence['day'].apply(lambda x: x if x > 25 else x+30) - 32)*(-1)
df = df.merge(first_banner_appearence[['banner_id', 'days_ago']], on='banner_id')

Подрежем фичи с большой cardinality - редкие фичи схлопнем

In [34]:
banner_counts = df['banner_id'].value_counts()
banner_counts = banner_counts[banner_counts > 100]
df.banner_id = df.banner_id.progress_apply(lambda x: x if x in(banner_counts.index) else -1)


100%|██████████| 15821472/15821472 [00:29<00:00, 534525.01it/s]


In [33]:
zone_popularity = df['zone_id'].value_counts()
zone_popularity = zone_popularity[zone_popularity>100]
df.zone_id = df.zone_id.progress_apply(lambda x: x if x in(zone_popularity.index) else -1)


100%|██████████| 15821472/15821472 [00:28<00:00, 547307.22it/s]


Некоторые фичи вовсе бесполезны

In [15]:
df['impressions'].value_counts()

1    15821472
Name: impressions, dtype: int64

In [16]:
df = df.drop(['impressions'], axis=1)

### Интеракции

In [7]:
df['zone_banner'] = df['zone_id'].astype('str') + df['banner_id'].astype('str')
df['zone_os'] = df['zone_id'].astype('str') + df['os_id'].astype('str')
df['country_hour'] = df['country_id'].astype('str') + df['hour'].astype('str')
df['weekday_hour'] = df['hour'].astype('str') + df['weekday'].astype('str')

In [ ]:
# тоже обрезает непопулярные значения
def only_popular_values(df, col):
    zone_banner = df.zone_banner.value_counts()
    zone_banner = zone_banner[zone_banner > 100]
    df.zone_banner = df.zone_banner.progress_apply(lambda x: x if x in(zone_banner.index) else -1)
    return df

for col in ['zone_banner', 'zone_os', 'country_hour', 'weekday_hour']:
    df = only_popular_values(df, col)

In [17]:
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

df_train, df_test = df[df['day'] != 2], df[df['day'] == 2]
X_train = df_train.drop(['clicks', 'date_time'], axis=1)
y_train = df_train['clicks'].values
X_test = df_test.drop(['clicks', 'date_time'], axis=1)
y_test = df_test['clicks'].values

### Baseline

Будем предсказывать распределение 1 и 0 в кликах в обучающем множестве

In [42]:
y_pred_zero_proba = y_train[y_train == 0].shape[0] / y_train.shape[0]
y_pred_ones_proba = y_train[y_train == 1].shape[0] / y_train.shape[0]

print("Log-loss by predicting class distribution of train dataset:")

log_loss(y_test, [[y_pred_zero_proba, y_pred_ones_proba]]*len(y_test))

Log-loss by predicting class distribution of train dataset:


0.15486198009919713

In [39]:
from category_encoders import TargetEncoder
from sklearn.metrics import log_loss, make_scorer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GroupShuffleSplit

def regressor_pipeline(X, y):
    categorical_te_features = ['zone_banner', 'zone_os', 'country_hour', 'weekday_hour', 
                               'os_id', 'country_id', 'zone_id', 'banner_id', 'hour', 'weekday', 'day']
    num_features = ['campaign_clicks']
    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())])

    te = TargetEncoder(cols = categorical_te_features).fit(X[categorical_te_features], y)
    preprocessor = ColumnTransformer(
        transformers=[
            ('num_t', numeric_transformer, num_features),
            ('te', te, categorical_te_features)
            ],
        remainder='passthrough')
    
    regressor = LogisticRegression(penalty='l2', solver='liblinear', class_weight='balanced', random_state=42)
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', regressor)])


    param_grid = {
        'regressor__C': [1e-2, 1e-4],
    }
    # также для валидации по дням
    groups = X['day'].values

    grid_search = GridSearchCV(pipeline,
                               param_grid,
                               scoring='neg_log_loss',
                               n_jobs=-1,
                               cv=GroupShuffleSplit(n_splits=3, train_size=1, random_state=42).split(X, y, groups),
                               verbose = 10)
    grid_search.fit(X, y)
    print(f'Best parameters: {grid_search.best_params_}\nCV score={grid_search.best_score_})')
    return grid_search.best_score_, grid_search, X.shape[0]

In [40]:
ans, model, samples_count = regressor_pipeline(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


/Users/e.poslovskaya/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best parameters: {'regressor__C': 0.01}
CV score=-0.6048216908638393)


In [41]:
pred = model.predict_proba(X_test)
log_loss(y_test, pred)

0.7357687942663552

In [ ]:
# бейзлайн не побит :с но такого пункта не было в задании:)